# Testing out Linear Programming

In [ ]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value

In [ ]:
path = "../data/parts/"

In [ ]:
# Load data using pandas
cpus = pd.read_csv(path + "CPU_Data(new).csv")
gpus = pd.read_csv(path + "GPU_Data.csv")
rams = pd.read_csv(path + "RAM_Data.csv")

motherboards = pd.read_csv(path + "MOBO_Data(new2).csv")
psus = pd.read_csv(path + "PSU_Data.csv")
cases = pd.read_csv(path + "Case_Data.csv")
storage = pd.read_csv(path + "Storage_Data.csv")

games = pd.read_csv("../data/games/top300.csv")

# Return original value if conversion fails
def safe_to_numeric(val):
    try:
        return pd.to_numeric(val)
    except (ValueError, TypeError):
        return val  

# Convert numeric columns from string to numeric types
cpus = cpus.apply(safe_to_numeric)
gpus = gpus.apply(safe_to_numeric)
rams = rams.apply(safe_to_numeric)
cases = cases.apply(safe_to_numeric)
motherboards = motherboards.apply(safe_to_numeric)
psus = psus.apply(safe_to_numeric)
storage = storage.apply(safe_to_numeric)
games = games.apply(safe_to_numeric)

In [ ]:
# selected_game = 'No Man's Sky'
# selected_game = 'ELDEN RING'
# selected_game = 'Red Dead Redemption 2'
# selected_game = 'Cyberpunk 2077'

# budget = 2000
# budget = 2500
# budget = 3000

selected_game = 'Counter-Strike 2'
budget = 2500

# Select the game (copy a game name from top100.csv)
game_data = games[games['name'] == selected_game].iloc[0]

In [ ]:
#Filtering 

# Available brands for each part
# CPU: AMD, Intel
# GPU: AMD, ASRock, Asus, EVGA, Gainward, Gigabyte, Intel, MSI, Nvidia, PNY, PowerColor, Sapphire, Zotac
# Ram : Kingston, XPG, Fanxiang, PNY, Corsair, Samsung, Apacer, KLEVV
# Motherboard (Mobo): ASUS, MSI, ASRock, GIGABYTE, Colorful, NZXT
# PSU: 1st Player, Asus, Cooler, Deepcool, Gigabyte, MSI, Silverstone, Thermaltake
# Case: Armaggeddon, 1st Player, Lian Li

# Define allowed brands (customize as needed)
allowed_cpu_brands = ["Intel"]
allowed_gpu_brands = ["Nvidia", "Asus", "Gainward"]
allowed_ram_brands = ["Kingston", "Corsair"]
allowed_psu_brands = ["1st Player", "ASRock"]
allowed_mobo_brands = ["MSI", "ASUS"]
allowed_case_brands = ["1st Player"]

# Filter each DataFrame by brand
cpus_filtered = cpus[cpus["Brand"].isin(allowed_cpu_brands)].reset_index(drop=True)
gpus_filtered = gpus[gpus["Brand"].isin(allowed_gpu_brands)].reset_index(drop=True)
rams_filtered = rams[rams["Brand"].isin(allowed_ram_brands)].reset_index(drop=True)
psus_filtered = psus[psus["Brand"].isin(allowed_psu_brands)].reset_index(drop=True)
motherboards_filtered = motherboards[motherboards["Brand"].isin(allowed_mobo_brands)].reset_index(drop=True)
cases_filtered = cases[cases["Brand"].isin(allowed_case_brands)].reset_index(drop=True)

# Validation to ensure filtered lists are not empty
if cpus.empty or gpus.empty or rams.empty or psus.empty or motherboards.empty or cases.empty:
    raise ValueError("One or more component categories have no items after filtering. Adjust brand filters.")

In [ ]:
# stuff to make it easy to pick between filtered and non filtered parts
# and to force the min ram to be 16 gb since that is the standard for having a pc

filter = True
min_ram_16 = True

if filter == True:
    lenCPU = range(len(cpus_filtered))
    lenGPU = range(len(gpus_filtered))
    lenRAM = range(len(rams_filtered))
    lenPSU = range(len(psus_filtered))
    lenMOBO = range(len(motherboards_filtered))
    lenCase = range(len(cases_filtered))

    cpusf = cpus_filtered
    gpusf = gpus_filtered
    ramf = rams_filtered
    psusf = psus_filtered
    mobosf = motherboards_filtered
    casesf = cases_filtered

else:
    lenCPU = range(len(cpus))
    lenGPU = range(len(gpus))
    lenRAM = range(len(rams))
    lenPSU = range(len(psus))
    lenMOBO = range(len(motherboards))
    lenCase = range(len(cases))

    cpusf = cpus
    gpusf = gpus
    ramsf = rams
    psusf = psus
    mobosf = motherboards
    casesf = cases

In [ ]:
# --- Create LP Problem ---
problem = LpProblem("Desktop_Optimization", LpMaximize)

# --- Define Variables ---
cpu_vars = [LpVariable(f"cpu_{i}", cat="Binary") for i in lenCPU]
gpu_vars = [LpVariable(f"gpu_{i}", cat="Binary") for i in lenGPU]
ram_vars = [LpVariable(f"ram_{i}", lowBound=0, upBound=2, cat="Integer") for i in lenRAM]
psu_vars = [LpVariable(f"psu_{i}", cat="Binary") for i in lenPSU]
mobo_vars = [LpVariable(f"mb_{i}", cat="Binary") for i in lenMOBO]
case_vars = [LpVariable(f"case_{i}", cat="Binary") for i in lenCase]

# --- Cost Function ---
total_cost = (
    lpSum(cpu_vars[i] * cpusf.iloc[i]["Price"] for i in lenCPU) +
    lpSum(gpu_vars[i] * gpusf.iloc[i]["Price"] for i in lenGPU) +
    lpSum(ram_vars[i] * ramsf.iloc[i]["Price"] for i in lenRAM) +
    lpSum(psu_vars[i] * psusf.iloc[i]["Price"] for i in lenPSU) +
    lpSum(mobo_vars[i] * mobosf.iloc[i]["Price"] for i in lenMOBO) +
    lpSum(case_vars[i] * casesf.iloc[i]["Price"] for i in lenCase)
)

# Minimize cost by maximizing (-cost)
problem += -total_cost  

# --- Constraints ---

# Budget
problem += total_cost <= budget, "Budget_Constraint"

# CPU and GPU performance
problem += (
    lpSum(cpu_vars[i] * cpusf.iloc[i]["Score"] for i in lenCPU) >= game_data["CPU"],
    "CPU_Performance_Constraint",
)
problem += (
    lpSum(gpu_vars[i] * gpusf.iloc[i]["Score"] for i in lenGPU) >= game_data["GPU"],
    "GPU_Performance_Constraint",
)

# RAM constraint (careful with MB to GB)
ram_requirement_gb = game_data["memory"]
if min_ram_16 and ram_requirement_gb < 16:
    ram_requirement_gb = 16  # Force minimum 16 GB RAM

problem += (
    lpSum(ram_vars[i] * ramsf.iloc[i]["Capacity (GB)"] for i in lenRAM) >= ram_requirement_gb,
    "RAM_Capacity_Constraint",
)

# Power constraint
problem += (
    lpSum(gpu_vars[i] * gpusf.iloc[i]["Recommended Power"] for i in lenGPU) <=
    lpSum(psu_vars[i] * psusf.iloc[i]["Wattage"] for i in lenPSU),
    "PSU_Power_Constraint",
)

# Only select exactly one of each major part
problem += lpSum(cpu_vars) == 1, "Select_One_CPU"
problem += lpSum(gpu_vars) == 1, "Select_One_GPU"
problem += lpSum(psu_vars) == 1, "Select_One_PSU"
problem += lpSum(mobo_vars) == 1, "Select_One_MOBO"
problem += lpSum(case_vars) == 1, "Select_One_Case"

# RAM quantity constraint: 1 or 2 sticks
problem += lpSum(ram_vars) >= 1, "At_Least_One_RAM"
problem += lpSum(ram_vars) <= 2, "At_Most_Two_RAM"

# --- Solve ---
problem.solve()

# --- Output Results ---

# Get selected components
selected_cpu_idx = [i for i in lenCPU if cpu_vars[i].value() == 1][0]
selected_gpu_idx = [i for i in lenGPU if gpu_vars[i].value() == 1][0]
selected_psu_idx = [i for i in lenPSU if psu_vars[i].value() == 1][0]
selected_mobo_idx = [i for i in lenMOBO if mobo_vars[i].value() == 1][0]
selected_case_idx = [i for i in lenCase if case_vars[i].value() == 1][0]

selected_ram_idx = [i for i in lenRAM if ram_vars[i].value() > 0]
selected_ram_count = ram_vars[selected_ram_idx[0]].value()
selected_ram_model = ramsf.iloc[selected_ram_idx[0]]['Model']
selected_ram_brand = ramsf.iloc[selected_ram_idx[0]]['Brand']

# Part names
selected_cpu = cpusf.iloc[selected_cpu_idx]['Name']
selected_gpu = gpusf.iloc[selected_gpu_idx]['Name']
selected_psu = psusf.iloc[selected_psu_idx]['Name']
selected_mobo = mobosf.iloc[selected_mobo_idx]['Name']
selected_case = casesf.iloc[selected_case_idx]['Model']

# Scores
selected_cpu_score = cpusf.iloc[selected_cpu_idx]["Score"]
selected_gpu_score = gpusf.iloc[selected_gpu_idx]["Score"]

# Prices
cpu_price = cpusf.iloc[selected_cpu_idx]["Price"]
gpu_price = gpusf.iloc[selected_gpu_idx]["Price"]
psu_price = psusf.iloc[selected_psu_idx]["Price"]
mobo_price = mobosf.iloc[selected_mobo_idx]["Price"]
case_price = casesf.iloc[selected_case_idx]["Price"]
ram_price = ramsf.iloc[selected_ram_idx[0]]["Price"] * selected_ram_count

# Calculate Total RAM capacity
total_ram_capacity = selected_ram_count * ramsf.iloc[selected_ram_idx[0]]["Capacity (GB)"]

# Total cost manually
total_cost_manual = cpu_price + gpu_price + ram_price + psu_price + mobo_price + case_price

# --- Final Display ---
print(f"Requirements for {selected_game}: ")
print(f"CPU Benchmark: {game_data['CPU']}")
print(f"GPU Benchmark: {game_data['GPU']}")
print(f"Memory: {game_data['memory']} GB\n")

print(f"Selected CPU: {selected_cpu}")
print(f"CPU Price: RM {cpu_price}")
print(f"CPU Benchmark Score: {selected_cpu_score}\n")

print(f"Selected GPU: {selected_gpu}")
print(f"GPU Price: RM {gpu_price}")
print(f"GPU Benchmark Score: {selected_gpu_score}\n")

print(f"Selected RAM Model: {selected_ram_brand} - {selected_ram_model}")
print(f"RAM Count: {int(selected_ram_count)}")
print(f"Total RAM Price: RM {ram_price}")
print(f"Total RAM Capacity: {total_ram_capacity} GB\n")

print(f"Selected PSU: {selected_psu}")
print(f"PSU Price: RM {psu_price}\n")

print(f"Selected Motherboard: {selected_mobo}")
print(f"Motherboard Price: RM {mobo_price}\n")

print(f"Selected Case: {selected_case}")
print(f"Case Price: RM {case_price}\n")

print(f"Total Cost: RM {total_cost_manual:.2f}")

print("\n❗ Keep in mind that hardware prices fluctuate often and that this list may not be accurate ❗")

In [ ]:
# old algo (just in case the new one breaks for some reason)


# Select the game (copy a game name from top100.csv)
game_data = games[games['name'] == selected_game].iloc[0]

# Create the LP problem
problem = LpProblem("Desktop_Optimization", LpMaximize)

# Define variables
cpu_vars = [LpVariable(f"cpu_{i}", cat="Binary") for i in range(len(cpus))]
gpu_vars = [LpVariable(f"gpu_{i}", cat="Binary") for i in range(len(gpus))]
ram_vars = [LpVariable(f"ram_{i}", lowBound=0, upBound=2, cat="Integer") for i in range(len(rams))]  # Variable for selecting 1 or 2 identical RAM modules
psu_vars = [LpVariable(f"psu_{i}", cat="Binary") for i in range(len(psus))]

# Objective function: Minimize cost
problem += (
    lpSum(cpu_vars[i] * cpus.iloc[i]["Price"] for i in range(len(cpus))) +
    lpSum(gpu_vars[i] * gpus.iloc[i]["Price"] for i in range(len(gpus))) +
    lpSum(ram_vars[i] * rams.iloc[i]["Price"] for i in range(len(rams))) +
    lpSum(psu_vars[i] * psus.iloc[i]["Price"] for i in range(len(psus))),
    "Total_Cost",
)

# Budget constraint
problem += (
    lpSum(cpu_vars[i] * cpus.iloc[i]["Price"] for i in range(len(cpus))) +
    lpSum(gpu_vars[i] * gpus.iloc[i]["Price"] for i in range(len(gpus))) +
    lpSum(ram_vars[i] * rams.iloc[i]["Price"] for i in range(len(rams))) +
    lpSum(psu_vars[i] * psus.iloc[i]["Price"] for i in range(len(psus)))
    <= budget,
    "Budget_Constraint",
)

# Performance benchmarks for CPU and GPU
problem += (
    lpSum(cpu_vars[i] * cpus.iloc[i]["Score"] for i in range(len(cpus))) >= game_data['CPU'],
    "CPU_Performance_Constraint",
)
problem += (
    lpSum(gpu_vars[i] * gpus.iloc[i]["Score"] for i in range(len(gpus))) >= game_data['GPU'],
    "GPU_Performance_Constraint",
)

# RAM Capacity Constraint (ensuring it has enough capacity for the game)
# Mutliply by 1024 because capacity is in GB no MB in RAM_Data.csv
problem += (
    lpSum(ram_vars[i] * rams.iloc[i]["Capacity (GB)"] for i in range(len(rams))) >= game_data['memory'],  
    "RAM_Capacity_Constraint",
)

# Power constraint: Ensure PSU meets the recommended power for the selected GPU
problem += (
    lpSum(gpu_vars[i] * gpus.iloc[i]["Recommended Power"] for i in range(len(gpus))) <= 
    lpSum(psu_vars[i] * psus.iloc[i]["Wattage"] for i in range(len(psus))),
    "PSU_Power_Constraint",
)

# Ensure at most 2 identical RAM are picked
problem += lpSum(ram_vars) >= 1, "Min_One_RAM"
problem += lpSum(ram_vars) <= 2, "Max_Two_RAM"

# Only one CPU, GPU, and PSU can be selected
problem += lpSum(cpu_vars) == 1, "Select_One_CPU"
problem += lpSum(gpu_vars) == 1, "Select_One_GPU"
problem += lpSum(ram_vars) == 1, "Select_One_RAM"
problem += lpSum(psu_vars) == 1, "Select_One_PSU"

# Solve the problem
problem.solve()

# Output the results
selected_cpu = cpus.iloc[[i for i in range(len(cpus)) if cpu_vars[i].value() == 1]]["Name"].values[0]
selected_gpu = gpus.iloc[[i for i in range(len(gpus)) if gpu_vars[i].value() == 1]]["Name"].values[0]
selected_psu = psus.iloc[[i for i in range(len(psus)) if psu_vars[i].value() == 1]]["Name"].values[0]

# Find the RAM model where value > 0 (either 1 or 2 copies)
selected_ram_model_index = [i for i in range(len(rams)) if ram_vars[i].value() > 0]
if selected_ram_model_index:
    selected_ram_model = rams.iloc[selected_ram_model_index[0]]["Model"]
    selected_ram_count = int(ram_vars[selected_ram_model_index[0]].value())
else:
    selected_ram_model = None
    selected_ram_count = 0

# Get the benchmarks for selected CPU and GPU
selected_cpu_score = cpus.iloc[[i for i in range(len(cpus)) if cpu_vars[i].value() == 1]]["Score"].values[0]
selected_gpu_score = gpus.iloc[[i for i in range(len(gpus)) if gpu_vars[i].value() == 1]]["Score"].values[0]


# Calculate total cost
total_cost = (
    sum(cpu_vars[i].value() * cpus.iloc[i]["Price"] for i in range(len(cpus))) +
    sum(gpu_vars[i].value() * gpus.iloc[i]["Price"] for i in range(len(gpus))) +
    sum(ram_vars[i].value() * rams.iloc[i]["Price"] for i in range(len(rams))) +
    sum(psu_vars[i].value() * psus.iloc[i]["Price"] for i in range(len(psus)))
)

# Calculate the total RAM capacity
total_ram_capacity = selected_ram_count * rams.iloc[selected_ram_model_index[0]]["Capacity (GB)"] * 1024

# Display the results
print(f"Required CPU Benchmark for {selected_game}: {game_data['CPU']}")
print(f"Required GPU Benchmark for {selected_game}: {game_data['GPU']}")
print(f"Required Memory for {selected_game}: {game_data['memory']} MB\n")

print(f"Selected CPU: {selected_cpu}")
print(f"CPU Benchmark Score: {selected_cpu_score}\n")
print(f"Selected GPU: {selected_gpu}")
print(f"GPU Benchmark Score: {selected_gpu_score}\n")

# Display the selected RAM model and its count (either 1 or 2 copies)
print(f"Selected RAM Model: {selected_ram_model}")
print(f"Selected RAM Count: {selected_ram_count}")
print(f"Total RAM Capacity: {total_ram_capacity} MB\n")
print(f"Selected PSU: {selected_psu}")
print(f"Total Cost: {total_cost}")